### Use case notebook: schedule CCB team meetings ###
Date: 03/06/2025

In [1]:
import os
import glob
import numpy as np
from collections import deque

# There is a warning in the timeboard library that we want to suppress here
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from pathlib import Path
import time
import timeboard as tb
import timeboard.calendars.US as US
import datetime
import holidays

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import the package
%load_ext autoreload
%autoreload 2
import cadence
from cadence.utils import FileOP
from cadence.mscheduler import Meetings
from cadence.mscheduler import cyclic_permutate
print(f'Package version: {cadence.__version__}')

Package version: unknown


### Meeting participants and settings ###

In [2]:
# Docker drive
data_dir = os.path.join(os.environ.get('HOME'), 'data')

group_members_file_name = 'CCB_Team_Jan25.xlsx'
member_file = os.path.join(data_dir, group_members_file_name)

tm_raw = pd.read_excel(member_file)
display(tm_raw)

,name,email,group
0,Andreas,andreas_werdich@hms.harvard.edu,ai-data
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data
2,Nathan,nathan_palmer@hms.harvard.edu,ai-data
3,Alex,alex_pickering@hms.harvard.edu,comp-bio
4,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio
5,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio
6,Grey,grey_kuling@hms.harvard.edu,ai-data
7,Ludwig,ludwig_geistlinger@hms.harvard.edu,comp-bio
8,Tram,tram_nguyen@hms.harvard.edu,comp-bio
9,Tyrone,tyrone_lee@hms.harvard.edu,comp-bio


### Cleanup of the spreadsheet ###
Just in case that was not done. To provide a consistent look.

In [3]:
email_col = 'email'
name_col = 'name'
group_col = 'group'

# Lets do some cleanup of the spreadsheet
# small letters for the column names
tm = tm_raw.copy()
tm.columns = [col.lower() for col in tm.columns]

# small letters for emails and group, first letter capitalize for names
tm[email_col] = tm[email_col].str.lower()
tm[group_col] = tm[group_col].str.lower()
tm[name_col] = tm[name_col].str.title()

# Sort by group and then name
tm = tm.sort_values(by=['group', 'name'], ascending=True).reset_index(drop=True)

# Mark names that are not presenting
rm_name = ['nathan', 'ludwig']
rm_name = [nm.title() for nm in rm_name]
tm = tm.assign(presenting=True)
tm.loc[tm['name'].isin(rm_name), 'presenting'] = False

# save the name ist as a csv_file
presenter_list_file_name = f'{os.path.splitext(group_members_file_name)[0]}_list.csv'
presenter_list_file = os.path.join(data_dir, presenter_list_file_name)
tm.to_csv(presenter_list_file, index=False)
display(tm)

,name,email,group,presenting
0,Andreas,andreas_werdich@hms.harvard.edu,ai-data,True
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,True
2,Grey,grey_kuling@hms.harvard.edu,ai-data,True
3,Nathan,nathan_palmer@hms.harvard.edu,ai-data,False
4,Alex,alex_pickering@hms.harvard.edu,comp-bio,True
5,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio,True
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,True
7,Ludwig,ludwig_geistlinger@hms.harvard.edu,comp-bio,False
8,Tram,tram_nguyen@hms.harvard.edu,comp-bio,True
9,Tyrone,tyrone_lee@hms.harvard.edu,comp-bio,True


In [4]:
# Remove the names that we do not want in the schedule
presenters_df = tm.loc[tm['presenting']==True]
display(presenters_df)

,name,email,group,presenting
0,Andreas,andreas_werdich@hms.harvard.edu,ai-data,True
1,Gerald,gerald_kiwanuka@hms.harvard.edu,ai-data,True
2,Grey,grey_kuling@hms.harvard.edu,ai-data,True
4,Alex,alex_pickering@hms.harvard.edu,comp-bio,True
5,Andrew,andrew_ghazi@hms.harvard.edu,comp-bio,True
6,Anthony,anthony-alexander_christidis@hms.harvard.edu,comp-bio,True
8,Tram,tram_nguyen@hms.harvard.edu,comp-bio,True
9,Tyrone,tyrone_lee@hms.harvard.edu,comp-bio,True


### Shuffle the data and create a presenter list ###

In [7]:
# Let's do a random shuffling of the data frame
random_state = 234
presenters_df_shuffled = presenters_df.sample(frac=1, random_state=random_state)

# Get the names and the groups from the new data frame
name_list = list(presenters_df_shuffled['name'].values)
group_list = list(presenters_df_shuffled['group'].values)

# Instantiate the Meetings class with the list of names and groups from the data frame
meet = Meetings(name_list=name_list, group_list=group_list)

# We create a presenter list by merging the groups so that we have a member 
# from a different group presenting each time
name_sequence = meet.create_name_sequence()
# Rotate the sequence (start with a specific name)
rotated_sequence = cyclic_permutate(name_sequence, name='Grey')
updated_sequence = ['Grey', 'Andrew','Alex', 'Gerald', 'Tram', 'Andreas', 'Anthony', 'Tyrone']
new_presenter_list = meet.create_name_sequence(name_sequence=updated_sequence, merge_groups=False)
display(meet.name_df)

,name,group
0,Grey,ai-data
1,Andrew,comp-bio
2,Alex,comp-bio
3,Gerald,ai-data
4,Tram,comp-bio
5,Andreas,ai-data
6,Anthony,comp-bio
7,Tyrone,comp-bio


### Create the meeting schedule ###

In [11]:
start_date = '2025-03-05'
end_date = '2025-06-25'

cal = meet.create_timeboard(start_date=start_date, 
                            end_date=end_date,
                            start_name='Gerald')
skip_date = '2025-03-12'
skip_name = 'CCB'
skip_comment = 'Presentation brainstorming for retreat 04/02'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

skip_date = '2025-04-02'
skip_name = 'CCB'
skip_comment = 'Retreat'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

display(cal)

,date,name,group,holiday,comment
0,2025-03-05,Gerald,ai-data,False,None
1,2025-03-12,CCB,comp-bio,False,Presentation brainstorming for retreat 04/02
2,2025-03-19,Tram,comp-bio,False,None
3,2025-03-26,Andreas,ai-data,False,None
4,2025-04-02,CCB,comp-bio,False,Retreat
5,2025-04-09,Anthony,comp-bio,False,None
6,2025-04-16,Tyrone,comp-bio,False,None
7,2025-04-23,Grey,ai-data,False,None
8,2025-04-30,Andrew,comp-bio,False,None
9,2025-05-07,Alex,comp-bio,False,None


In [12]:
# Save the presentation schedule
save_cols = ['date', 'name', 'comment']
schedule_name = f'ccb_presentations_2025_03.csv'
schedule_file = os.path.join(data_dir, schedule_name)
cal[save_cols].to_csv(schedule_file, index=False)
display(cal[save_cols])
print(schedule_file)

,date,name,comment
0,2025-03-05,Gerald,None
1,2025-03-12,CCB,Presentation brainstorming for retreat 04/02
2,2025-03-19,Tram,None
3,2025-03-26,Andreas,None
4,2025-04-02,CCB,Retreat
5,2025-04-09,Anthony,None
6,2025-04-16,Tyrone,None
7,2025-04-23,Grey,None
8,2025-04-30,Andrew,None
9,2025-05-07,Alex,None


/app/data/ccb_presentations_2025_03.csv
